## Tutorial showing use of a `Workspace` object
### Part 1: making tables and plots
PyGSTi's `Workspace` object can be used on a variety of levels.  At the most basic level, it can be used as a plot and/or table factory which generates nice output based on quantities (e.g. GateSets, DataSets, etc.) that you've computed or loaded within a notebook.  For this, it's useful to call `init_notebook_mode` with `autodisplay=True` so that you don't have to `.display()` everything - `display()` gets called automatically when a plot or table is created.

First, let's run GST on the standard 1Q gate set to get some results to play with.

In [1]:
import numpy as np
import pygsti
from pygsti.construction import std1Q_XYI

In [2]:
#The usual GST setup: we're going to run GST on the standard XYI 1-qubit gateset
gs_target = std1Q_XYI.gs_target
fiducials = std1Q_XYI.fiducials
germs = std1Q_XYI.germs
maxLengths = [1,2,4,8]
listOfExperiments = pygsti.construction.make_lsgst_experiment_list(
    gs_target.gates.keys(), fiducials, fiducials, germs, maxLengths)

In [3]:
#Create some datasets for analysis
gs_datagen1 = gs_target.depolarize(gate_noise=0.1, spam_noise=0.001)
gs_datagen2 = gs_target.depolarize(gate_noise=0.05, spam_noise=0.01).rotate(rotate=0.01)

ds1 = pygsti.construction.generate_fake_data(gs_datagen1, listOfExperiments, nSamples=1000,
                                            sampleError="binomial", seed=1234)
ds2 = pygsti.construction.generate_fake_data(gs_datagen2, listOfExperiments, nSamples=1000,
                                            sampleError="binomial", seed=1234)
ds3 = ds1.copy_nonstatic(); ds3.add_counts_from_dataset(ds2); ds3.done_adding_data()

In [4]:
#Run GST on all three datasets
gs_target.set_all_parameterizations("TP")
results1 = pygsti.do_long_sequence_gst(ds1, gs_target, fiducials, fiducials, germs, maxLengths, verbosity=0)
results2 = pygsti.do_long_sequence_gst(ds2, gs_target, fiducials, fiducials, germs, maxLengths, verbosity=0)
results3 = pygsti.do_long_sequence_gst(ds3, gs_target, fiducials, fiducials, germs, maxLengths, verbosity=0)

#make some shorthand variable names for later
tgt = results1.gatesets['target']

ds1 = results1.dataset
ds2 = results2.dataset
ds3 = results3.dataset

gs1 = results1.gatesets['final estimate']
gs2 = results2.gatesets['final estimate']
gs3 = results3.gatesets['final estimate']

gss = results1.gatestring_structs['final']

Now that we have some results, let's create a `Workspace` and make some plots and tables.  Note that you can start typing "`w.`" and TAB-complete to see the different things a `Workspace` can make for you.  Furthermore, pressing SHIFT-TAB after the opening parenthesis of a function,  e.g. after typing "`w.GatesVsTargetTable(`", will bring up Jupyter's help window showing you the function signature (the arguments you need to give the function).

In [5]:
from pygsti.report import workspace
w = workspace.Workspace()
w.init_notebook_mode(connected=True, autodisplay=True) #so that displaying within a notebook works

In [6]:
w.GatesVsTargetTable(gs1, tgt)

Gate,Process Infidelity,1/2 Trace Distance,1/2 Diamond-Norm
Gi,0.074758,0.074836,0.074856
Gx,0.07094,0.070941,0.070941
Gy,0.076925,0.076929,0.076933


In [7]:
w.SpamVsTargetTable(gs2, tgt)

Prep/POVM,State Infidelity,1/2 Trace Distance
ρ0,0.005372,0.007159
E0,0.004795,0.005624
EC,0.004991,0.005624


In [8]:
w.ColorBoxPlot(("chi2","logl"), gss, ds1, gs1, boxLabels=True)
  #Notice how long it takes to switch between "chi2" and "logl".  This 
  # is due to drawing all of the box labels (boxLabels=True).

In [9]:
#This one requires knowng that each Results object holds a list of gatesets
# from each GST intation along with the corresponding gate strings that were used.
w.FitComparisonTable(gss.Ls, results1.gatestring_structs['iteration'],
                     results1.gatesets['iteration estimates'], ds1,)

L,2Δ(log L),k,2Δ(log L)-k,√2k,Nsigma,Ns,Np,Rating
1,86.45386,61,25.45386,11.04536,2.3,92,31,★★★★
2,180.5448,137,43.54481,16.55295,2.63,168,31,★★★★
4,472.3175,410,62.31748,28.63564,2.18,441,31,★★★★
8,830.1101,786,44.11009,39.64846,1.11,817,31,★★★★


In [10]:
# We can reuse 'gss' for all three since the gate sequences are the same.
w.FitComparisonTable(["GS1","GS2","GS3"], [gss, gss, gss], [gs1,gs2,gs3], ds1, Xlabel="GateSet")

GateSet,2Δ(log L),k,2Δ(log L)-k,√2k,Nsigma,Ns,Np,Rating
GS1,830.1101,786,44.11009,39.64846,1.11,817,31,★★★★
GS2,19603.31,786,18817.31,39.64846,5×102,817,31,★
GS3,5040.289,786,4254.289,39.64846,1×102,817,31,★★


In [11]:
w.ChoiTable(gs3)

Gate,Choi matrix (Pauli basis),Eigenvalues,Eigenvalue Magnitudes
Gi,$ \left(\!\!\begin{array}{cccc} 0.944898 & 0.002521e^{i1.588} & 0.003836e^{i1.446} & 0.003997e^{i1.544} \\ 0.002521e^{-i1.588} & 0.01893 & 0.000718e^{-i0.150} & 0.000839e^{i2.537} \\ 0.003836e^{-i1.446} & 0.000718e^{i0.150} & 0.018952 & 0.000203e^{i2.927} \\ 0.003997e^{-i1.544} & 0.000839e^{-i2.537} & 0.000203e^{-i2.927} & 0.01722 \end{array}\!\!\right) $,$ \begin{array}{c} 0.01685 \\ 0.018353 \\ 0.019859 \\ 0.944938 \end{array} $,
Gx,$ \left(\!\!\begin{array}{cccc} 0.480523 & 0.464377e^{i1.571} & 0.000865e^{i1.561} & 0.001272e^{i1.553} \\ 0.464377e^{-i1.571} & 0.486466 & 0.002446e^{-i0.009} & 0.001273e^{i3.135} \\ 0.000865e^{-i1.561} & 0.002446e^{i0.009} & 0.015203 & 0.000143e^{-i1.189} \\ 0.001272e^{-i1.553} & 0.001273e^{-i3.135} & 0.000143e^{i1.189} & 0.017809 \end{array}\!\!\right) $,$ \begin{array}{c} 0.014787 \\ 0.016806 \\ 0.020521 \\ 0.947886 \end{array} $,
Gy,$ \left(\!\!\begin{array}{cccc} 0.478634 & 0.00294e^{i1.565} & 0.463288e^{i1.571} & 0.001289e^{i1.588} \\ 0.00294e^{-i1.565} & 0.018929 & 0.000372e^{i0.060} & 0.000273e^{-i1.395} \\ 0.463288e^{-i1.571} & 0.000372e^{-i0.060} & 0.485058 & 0.001293e^{-i3.128} \\ 0.001289e^{-i1.588} & 0.000273e^{i1.395} & 0.001293e^{i3.128} & 0.017379 \end{array}\!\!\right) $,$ \begin{array}{c} 0.015635 \\ 0.018148 \\ 0.021066 \\ 0.945152 \end{array} $,


In [12]:
w.GateMatrixPlot(gs1['Gx'],scale=1.5, boxLabels=True)
w.GateMatrixPlot(pygsti.tools.error_generator(gs1['Gx'], tgt['Gx']), scale=1.5)

In [13]:
w.ErrgenTable(gs3,tgt)

Gate,Error Generator,Hamiltonian Projections,Stochastic Projections
Gi,,,
Gx,,,
Gy,,,


In [14]:
w.PolarEigenvaluePlot([np.linalg.eigvals(gs2['Gx'])],["purple"],scale=1.5)

In [15]:
w.GateEigenvalueTable(gs2, display=('evals','polar'))

Gate,Eigenvalues,Eigenvalues
Gi,$ \begin{array}{cc} 0.948655e^{i0.022} & 0.948655e^{-i0.022} \\ 0.948031 & 1 \end{array} $,
Gx,$ \begin{array}{cc} 1 & 0.951515 \\ 0.950678e^{i1.580} & 0.950678e^{-i1.580} \end{array} $,
Gy,$ \begin{array}{cc} 1 & 0.949877 \\ 0.950367e^{i1.581} & 0.950367e^{-i1.581} \end{array} $,
